<a href="https://colab.research.google.com/github/GHMelany/AMD_project/blob/main/AMD%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJECT: Finding similar items. Implement a detector of pairs of similar book reviews


LIBRARIES

In [1]:
!pip install pyspark

In [2]:
!pip install langid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=308a9fa1413d6d887c09bf04d4dd7e83bb3e0396545ceec2853292e834b94d35
  Stored in directory: /root/.cache/pip/wheels/3c/bc/9d/266e27289b9019680d65d9b608c37bff1eff565b001c977ec5
Successfully built langid


In [3]:
import re, html, langid
import os
import zipfile
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StringType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, NGram, HashingTF, MinHashLSH
from pyspark.ml import Pipeline


In [4]:
# ⚙️ Installa Java e Spark in Colab
!apt-get update
!apt-get install openjdk-11-jdk -y
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

# ⚙️ Configura le variabili d'ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages 

In [5]:
os.environ['KAGGLE_USERNAME'] = "melanygomez"
os.environ['KAGGLE_KEY'] = "38db1cce93622035560027022e9cafc"

!pip install -q kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:13<00:00, 24.6MB/s]
100% 1.06G/1.06G [00:13<00:00, 83.6MB/s]


In [6]:
zip_path = "amazon-books-reviews.zip"
extract_dir = "amazon_books_reviews"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(os.path.join(root, file))

folder = "amazon_books_reviews"
csv_path = os.path.join(folder, "Books_rating.csv")



amazon_books_reviews/Books_rating.csv
amazon_books_reviews/books_data.csv


In [7]:
spark = SparkSession.builder.appName("ReviewSimilarityPipeline").getOrCreate()

Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [ ]:
data = spark.read.csv(csv_path, header=True, inferSchema=True, quote='"', escape='"')
data = data.select("Id", F.col("review/score").alias("score"), F.col("review/text").alias("text")).dropna(subset=["text"])

In [ ]:
def preprocess(raw):
    if not raw: return ""
    txt = html.unescape(raw).lower()
    txt = re.sub(r"[^a-z0-9 ]+", " ", txt)
    return re.sub(r"\s+", " ", txt).strip()

def detect_lang(text):
    snippet = " ".join(text.split()[:50])
    lang, _ = langid.classify(snippet)
    return lang

def classify(score):
    if score > 3:
        return "positive"
    else:
        return "negative"


In [ ]:
clean_udf = F.udf(preprocess, StringType())
lang_udf = F.udf(detect_lang, StringType())
sentiment_udf = F.udf(classify, StringType())

In [ ]:
data = data.withColumn("clean_text", clean_udf("text")).dropDuplicates(["clean_text"])
data = data.withColumn("lang", lang_udf("clean_text")).filter(F.col("lang") == "en").drop("lang")
data = data.withColumn("sentiment", sentiment_udf("score"))


In [ ]:
tokenize = RegexTokenizer(inputCol="clean_text", outputCol="tokens", pattern="\\W+")
remove_sw = StopWordsRemover(inputCol="tokens", outputCol="content_words")
prep_pipeline = Pipeline(stages=[tokenize, remove_sw])
data = prep_pipeline.fit(data).transform(data)

In [ ]:
data = data.withColumn("length", F.size("content_words"))
data = data.filter((F.col("length") >= 20) & (F.col("length") <= 200))

In [ ]:
reviews = data.select("Id", "score", "sentiment", "content_words")

In [ ]:
vocab_size = reviews.select(F.explode("content_words")).distinct().count()
hash_space = 2 ** vocab_size.bit_length()

In [ ]:
def find_similar_pairs(input_df, shingle_len=3, threshold=0.8, hash_tables=40):
    ngram_gen = NGram(n=shingle_len, inputCol="content_words", outputCol="shingles")
    df_shingled = ngram_gen.transform(input_df)

    tf = HashingTF(inputCol="shingles", outputCol="vector", numFeatures=hash_space, binary=True)
    vectorized = tf.transform(df_shingled)

    minhash = MinHashLSH(inputCol="vector", outputCol="signature", numHashTables=hash_tables)
    model = minhash.fit(vectorized)
    transformed = model.transform(vectorized)

    candidates = model.approxSimilarityJoin(transformed, transformed, distCol="jaccard_dist", threshold=1 - threshold)
    similar = (candidates
               .filter(F.col("datasetA.Id") < F.col("datasetB.Id"))
               .withColumn("similarity", 1 - F.col("jaccard_dist"))
               .select(
                   F.col("datasetA.Id").alias("doc1"),
                   F.col("datasetB.Id").alias("doc2"),
                   "similarity"
               ))

    return similar


In [ ]:
pos_reviews = reviews.filter(F.col("sentiment") == "positive")
neg_reviews = reviews.filter(F.col("sentiment") == "negative")

pairs_pos = find_similar_pairs(pos_reviews, shingle_len=3, threshold=0.8)
pairs_neg = find_similar_pairs(neg_reviews, shingle_len=3, threshold=0.8)

In [ ]:
print(f"[✓] Coppie simili tra recensioni positive: {pairs_pos.count()}")
print(f"[✓] Coppie simili tra recensioni negative: {pairs_neg.count()}")

In [ ]:
print(f"[✓] Coppie simili tra recensioni positive: {pairs_pos.count()}")
print(f"[✓] Coppie simili tra recensioni negative: {pairs_neg.count()}")